In [2]:
import os
import pandas as pd
import numpy as np
import random
import subprocess
import shutil
import cv2

In [4]:
def copy_create(data_dataset_dir, split_name, name, i, data_dir):
    temp = ["csv","frame","video"]

    i1 = os.path.join(os.getcwd(), data_dataset_dir, split_name, i)
    if not os.path.exists(i1):
        os.makedirs(i1)
        print(f"create {i1}")
    for j in temp:
        j1 = os.path.join(i1, j)
        if not os.path.exists(j1):
            os.makedirs(j1)

        if j == "csv":
            source_file = os.path.join(data_dir, f"{i}.csv")
            destination_file = os.path.join(j1, f"{i}.csv")
            shutil.copy(source_file, destination_file)
        # if j == "frame":
        #     source_file = os.path.join(data_dir, f"{i}.csv")
        #     destination_file = os.path.join(j1, f"{i}.csv")
        #     shutil.copy(source_file, destination_file)
    return

In [5]:
def create_video(data_dataset_dir, split, i):
    path_video = os.path.join(data_dataset_dir, split, i, "video", f"{i}.mp4")
    image_folder = os.path.join(data_dataset_dir, split, i, "frame", i)
    images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]

# Sắp xếp các tệp tin ảnh theo thứ tự
    def extract_number(filename):
        # Tách các phần số từ tên tệp tin
        number_str = ''.join(filter(str.isdigit, filename))
        # Chuyển đổi chuỗi số thành số nguyên
        return int(number_str)
    images = sorted(images, key=extract_number)
    for i, img in enumerate(images):
        old_filename = os.path.join(image_folder, img)
        new_filename = os.path.join(image_folder, f"{i}.jpg")
        os.rename(old_filename, new_filename)
    new_images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
    new_images = sorted(new_images, key=extract_number)

    # Đọc chiều rộng và chiều cao của ảnh đầu tiên để cấu hình video
    frame = cv2.imread(os.path.join(image_folder, new_images[0]))
    height, width, layers = frame.shape

    # Tạo đối tượng VideoWriter
    video = cv2.VideoWriter(path_video, cv2.VideoWriter_fourcc(*'DIVX'), 1, (width, height))

    # Ghi các ảnh vào video
    for image in new_images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    # Đóng đối tượng VideoWriter
    cv2.destroyAllWindows()
    video.release()

    print("Video đã được tạo thành công!")
    print(path_video)
    return

In [ ]:
def main():
    root = os.getcwd()
    data_dir = 'data/GolfBall/Golf_Ball/Tracking/AnnotationsCsv'
    data_dataset_dir = 'data/GolfBall/Golf_Ball/Golf_Ball_Tracking_test'
    split = ["train","val"]

    name_video = []
    for i in os.listdir(data_dir):
        i = i.split(".")[0]
        name_video.append(i)
    print(name_video)

    len_split = int(len(name_video) - np.floor(len(name_video)/4))
    train_dir = name_video[0: len_split]
    val_dir = name_video[len_split:]

    i = 0
    for name in train_dir:
        copy_create(data_dataset_dir, split[0], name, i, data_dir)
        create_video(data_dataset_dir, split[0], name, i)
        i = i+1


    for name in val_dir:
        copy_create(data_dataset_dir, split[1], name, i, data_dir)
        create_video(data_dataset_dir, split[1], name, i)
        i = i+1
    return 